In [17]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
np.set_printoptions(suppress=True)

In [18]:
np.set_printoptions(suppress=True)
C = np.matrix([[16, 18, 9, 1],
    [13, 16, 18, 9],
    [14, 23, 12, 7]])
print(C)

[[16 18  9  1]
 [13 16 18  9]
 [14 23 12  7]]


In [19]:
a1=70
a2=50
a3=240

In [20]:
A = np.array([a1, a2, a3])

In [21]:
b1=100
b2=10
b3=70
b4=160

In [22]:
B = np.array([b1, b2, b3, b4])

In [23]:
N = len(A)
M = len(B)

In [24]:
df = pd.DataFrame(C, columns=B, index=A)
df.columns.name = 'aᵢ \\ bⱼ'
df

aᵢ \ bⱼ,100,10,70,160
70,16,18,9,1
50,13,16,18,9
240,14,23,12,7


In [25]:
df
if sum(A) == sum(B):
    print('Задача закрытого типа')
else:
    print('Задача открытого типа')

Задача открытого типа


In [26]:
P = np.zeros_like(C)
INF = 1_000_000_007
C_ = C.copy()
A_ = A.copy()
B_ = B.copy()
i = 0
j = 0

In [27]:
while not np.all(C_ == INF):
    i, j = np.unravel_index(C_.argmin(), C_.shape)
    if A_[i] <= B_[j]:
        C_[i].fill(INF)
        B_[j] -= A_[i]
        A_[i] = 0
    else:
        C_[:, j].fill(INF)
        A_[i] -= B[j]
        B_[j] = 0
if np.count_nonzero(P) != N + M - 1:
    print('Опорный план вырожденный')
else:
    print('Опорный план невырожденный')

Опорный план вырожденный


In [28]:
#A_ma1 - матрица коэфициентов
#x <= B_ma1 (E @ x <= A)
A_ma1 = np.array([[1 if i in range(j*M, M*(j+1)) else 0 for i in range(N*M)] for j in range(N)])
B_ma1 = A

#A_ma2 - матрица коэфициентов
#x = B_ma2 (E @ x = B)
A_ma2 = np.array([[1 if i % M == j else 0 for i in range(N*M)] for j in range(M)])
B_ma2 = B

res = linprog(C.flatten(), A_ma1, B_ma1, A_ma2, B_ma2, options={'disp': True})
X = res.x.round(3).reshape(N, M)
F_min = res.fun.round(3)

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 156.0               
0.2695701906645     0.2695701906645     0.2695701906645     0.73931049326    0.2695701906645     335.6235664662      
0.08047589481726    0.08047589481722    0.08047589481723    0.7222414541311  0.0804758948174     1066.36664975       
0.01244187161672    0.01244187161671    0.01244187161671    0.860039708023   0.01244187161675    2397.421064116      
0.002536855492748   0.002536855492747   0.002536855492747   0.8102648178974  0.002536855492743   2893.953603511      
2.023444286913e-06  2.023444286892e-06  2.023444286712e-06  0.9995189939702  2.02344428751e-06   3059.855698593      
1.011776363161e-10  1.011776487513e-10  1.011777554873e-10  0.9999499973151  1.011770611689e-10  3059.999992784      
Optimization terminated successfully.
         Current fu

In [29]:
print('Оптимальный план перевозки груза:')
X

Оптимальный план перевозки груза:


array([[ 0.,  0.,  0., 70.],
       [40., 10.,  0.,  0.],
       [60.,  0., 70., 90.]])

In [30]:
print('Оптимальная стоимость перевозки:')
F_min

Оптимальная стоимость перевозки:


3060.0